In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# rmdir /content/gdrive/MyDrive/Colab Notebooks/CMID/.ipynb_checkpoints

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

directory = "/content/gdrive/MyDrive/Colab Notebooks/CMID/"

train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                            #  label_mode = tf.float32,
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
i = 0
for images, labels in train_dataset: # train_dataset.take(1):
  # print(images.shape, labels.shape)
  if(i==0):
    X_train = images
    y_train = labels
  else:
    X_train = tf.concat([X_train, images], 0)
    y_train = tf.concat([y_train, labels], 0)

  i = i + 1
print(X_train.shape, y_train.shape)

i = 0
for images, labels in validation_dataset: # train_dataset.take(1):
  # print(images.shape, labels.shape)
  if(i==0):
    X_valid = images
    y_valid = labels
  else:
    X_valid = tf.concat([X_valid, images], 0)
    y_valid = tf.concat([y_valid, labels], 0)

  i = i + 1
print(X_valid.shape, y_valid.shape)

#CONCATENATING TRAINING AND VALIDATION IMAGES for GAN Model
X = tf.concat([X_train, X_valid], 0)
y = tf.concat([y_train, y_valid], 0)
print(X.shape, y.shape)

In [ ]:
#X_train for training the GAN model
flip = 0
for i in range(len(y)):
  if y[i] == 0: #0 for defective and 1 for non defective
    if flip == 0:
      X_train = [X[i]]
      flip=1
    else:
      X_train = tf.concat([X_train, [X[i]]], 0)
X_train = tf.concat([X_train, X_train, X_train], 0) # making it tripple
X_train.shape

In [ ]:
#normalizing the dataset
X_train = X_train / 255.0

n_H, n_W, n_C = [224, 224, 3]
n_classes =1
print(n_H, n_W, n_C )

In [ ]:
#use for AE, CAE, VAE
def show_reconstructions(model, images=X_valid, n_images=10):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    gan_loss = 1000
    for epoch in range(n_epochs):
        loss = []
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              # not shown in the book
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)

            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            temp_loss = gan.train_on_batch(noise, y2)
            loss.append(temp_loss)
        avg_loss = sum(loss)/len(loss) #avg loss on batches
        gan_loss = min(gan_loss, avg_loss)
        print('----loss=', avg_loss)
        # if gan_loss < 1:
        #   return gan_loss
    print('minimum gan_Loss=', gan_loss)
    return gan_loss

In [ ]:
import time
start_time = time.time()

#Deep Convolutional GAN
# codings_size = [2, 5, 10, 20, 25, 40, 50, 100, 150, 200]
codings_size = [100]
n_epochs=600
gan_loss = []
for i in range(len(codings_size)):
  print('\nCODINGS SIZE =', codings_size[i])
  generator = tf.keras.models.Sequential([
      tf.keras.layers.Dense(14 * 14 * 256, input_shape=[codings_size[i]]),
      tf.keras.layers.Reshape([14, 14, 256]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding="SAME",
                                  activation="selu"),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding="SAME",
                                  activation="selu"),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME",
                                  activation="selu"),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding="SAME",
                                  activation="tanh"),
      # keras.layers.Dense(n_H*n_W*n_C, activation="sigmoid"),
      # keras.layers.Reshape([n_H, n_W, n_C])
      # keras.layers.Conv2DTranspose(3, kernel_size=3, strides=2, padding="SAME", activation="sigmoid"),
      tf.keras.layers.Reshape([n_H, n_W, n_C])
  ])
  discriminator = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding="SAME",
                          activation=tf.keras.layers.LeakyReLU(0.2),
                          input_shape=[n_H, n_W, n_C]),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding="SAME",
                          activation=tf.keras.layers.LeakyReLU(0.2)),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding="SAME",
                          activation=tf.keras.layers.LeakyReLU(0.2)),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Conv2D(256, kernel_size=3, strides=2, padding="SAME",
                          activation=tf.keras.layers.LeakyReLU(0.2)),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1, activation="sigmoid") #ONE NEURON TO CLASSIFY IT IS REAL OR FAKE
  ])
  gan = tf.keras.models.Sequential([generator, discriminator])

  discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
  discriminator.trainable = False
  gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

  # print(X_train.shape)
  # X_train_dcgan = X_train.reshape(-1, n_H, n_W, n_C) * 2. - 1. # reshape and rescale
  X_train_dcgan = X_train

  batch_size = 32 #32
  dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
  dataset = dataset.shuffle(1000)
  dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

  loss = train_gan(gan, dataset, batch_size, codings_size[i], n_epochs)
  gan_loss.append(loss)
print("CODING SIZE =", codings_size)
print('GAN LOSS =', gan_loss)
print('minimum gan loss =', min(gan_loss), 'codings size=', codings_size[gan_loss.index(min(gan_loss))])
print(time.time() - start_time)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
codings_size = 100
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator(noise)
plot_multiple_images(generated_images, 8)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
for i in range(8):   #len(images.shape[0])
    ax = plt.subplot(4, 2, i + 1 )
    plt.imshow(generated_images[i]) #.numpy().astype("uint8"))
    # plt.title(class_names[labels[i]])
    plt.axis("off")

# Generate and save the new images

In [ ]:
bs = 200
for j in range(10):
  tf.random.set_seed(42)
  np.random.seed(42)
  noise = tf.random.normal(shape=[bs, codings_size])
  generated_images = generator(noise)
  plt.figure(figsize=(10, 10))
  for i in range(bs):
      plt.imshow(generated_images[i]) #.numpy().astype("uint8"))
      # plt.title(class_names[labels[i]])
      plt.axis("off")
      plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/generated1kEpochs/defective/generated_image'+str(j)+str(i)+'.jpg')
print(time.time() - start_time)